In [19]:
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday

# Load the CSV files
umsatz_df = pd.read_csv('umsatzdaten_gekuerzt.csv')
kiwo_df = pd.read_csv('kiwo.csv')
wetter_df = pd.read_csv('wetter.csv')

# Display the first few rows and column names to understand the structure
umsatz_df.head(), kiwo_df.head(), wetter_df.head()

(        id       Datum  Warengruppe      Umsatz
 0  1307011  2013-07-01            1  148.828353
 1  1307021  2013-07-02            1  159.793757
 2  1307031  2013-07-03            1  111.885594
 3  1307041  2013-07-04            1  168.864941
 4  1307051  2013-07-05            1  171.280754,
         Datum  KielerWoche
 0  2012-06-16            1
 1  2012-06-17            1
 2  2012-06-18            1
 3  2012-06-19            1
 4  2012-06-20            1,
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
 0  2012-01-01         8.0      9.8250                   14        58.0
 1  2012-01-02         7.0      7.4375                   12         NaN
 2  2012-01-03         8.0      5.5375                   18        63.0
 3  2012-01-04         4.0      5.6875                   19        80.0
 4  2012-01-05         6.0      5.3000                   23        80.0)

In [20]:
# Convert Datum columns to datetime format
umsatz_df['Datum'] = pd.to_datetime(umsatz_df['Datum'])
kiwo_df['Datum'] = pd.to_datetime(kiwo_df['Datum'])
wetter_df['Datum'] = pd.to_datetime(wetter_df['Datum'])

# Merge umsatz_df with kiwo_df
merged_df = pd.merge(umsatz_df, kiwo_df, on='Datum', how='left')

# Merge the result with wetter_df
merged_df = pd.merge(merged_df, wetter_df, on='Datum', how='left')

# Fill missing 'KielerWoche' and 'Wettercode' with 0 (assume not during Kieler Woche or no weather data)
merged_df['KielerWoche'] = merged_df['KielerWoche'].fillna(0)
merged_df['Wettercode'] = merged_df['Wettercode'].fillna(0)

# Add date-related features
merged_df['Wochentag'] = merged_df['Datum'].dt.day_name() # Day of the week
merged_df['Monat'] = merged_df['Datum'].dt.month
merged_df['Jahr'] = merged_df['Datum'].dt.year
merged_df['Kalenderwoche'] = merged_df['Datum'].dt.isocalendar().week




Adding Seasons

In [21]:
# Define seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Fruehling'
    elif month in [6, 7, 8]:
        return 'Sommer'
    else:
        return 'Herbst'

merged_df['Jahreszeit'] = merged_df['Monat'].apply(get_season)

Add Public Holiday Indicator

In [24]:

# Define German holidays
class GermanHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('Neujahr', month=1, day=1, observance=nearest_workday),
        Holiday('Tag der Arbeit', month=5, day=1, observance=nearest_workday),
        Holiday('Tag der Deutschen Einheit', month=10, day=3, observance=nearest_workday),
        Holiday('Erster Weihnachtstag', month=12, day=25, observance=nearest_workday),
        Holiday('Zweiter Weihnachtstag', month=12, day=26, observance=nearest_workday)
    ]


# Add holiday indicator
years = merged_df['Jahr'].unique()
start_date = f"{years.min()}-01-01"
end_date = f"{years.max()}-12-31"
calendar = GermanHolidayCalendar()
holidays = calendar.holidays(start=start_date, end=end_date)
merged_df['Feiertag'] = merged_df['Datum'].isin(holidays).astype(int)


In [23]:
merged_df.head()

,id,Datum,Warengruppe,Umsatz,KielerWoche,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Wochentag,Monat,Jahr,Kalenderwoche,Jahreszeit,Feiertag
0,1307011,2013-07-01,1,148.828353,0.0,6.0,17.8375,15.0,20.0,Monday,7,2013,27,Sommer,0
1,1307021,2013-07-02,1,159.793757,0.0,3.0,17.3125,10.0,0.0,Tuesday,7,2013,27,Sommer,0
2,1307031,2013-07-03,1,111.885594,0.0,7.0,21.0750,6.0,61.0,Wednesday,7,2013,27,Sommer,0
3,1307041,2013-07-04,1,168.864941,0.0,7.0,18.8500,7.0,20.0,Thursday,7,2013,27,Sommer,0
4,1307051,2013-07-05,1,171.280754,0.0,5.0,19.9750,12.0,0.0,Friday,7,2013,27,Sommer,0
